### Aprendizado Ensemble - Florestas Aleatórias

In [1]:
# bibliotecas utilizadas no desenvolvimento do projeto
from collections import Counter, defaultdict, OrderedDict
from functools import partial
import math, random
import pandas as pd
from csv import reader

In [5]:
def build_dataset(path_file): 
    dataset = list()
    with open(path_file, 'r') as file:
        csv_reader = reader(file)
        rows = [r for r in csv_reader]
    # capturando os cabeçalhos   
    headers = rows[0]
    
    remove_row_with_values_missing(rows, headers)
    
    for row in rows[1:len(rows)]:
        out_dic = {}
        for header_index in range(len(headers)):
            out_dic.update({ headers[header_index]: row[header_index]})
        
        # captura a classe
        class_key = list(out_dic.items())[-1]
        # remove a classe captura do dicionario
        del out_dic[list(out_dic.keys())[-1]]
        # montando o dataset de saída
        data_tuple = (out_dic, class_key[1])
        dataset.append(data_tuple)
    return dataset

# TODO: devemos remover linhas com dados faltando?
def remove_row_with_values_missing(rows, headers):
    quant = 0
    for row in rows[1:len(rows)]:
        for header_index in range(len(headers)):
            if(row[header_index] is None or row[header_index] == ""):
                quant = quant + 1
    

# construindo a base de dados a partir do arquivo fornecido
dataset = build_dataset('datasets/benchmark.csv')
for data in dataset:
    print(data)

({'Tempo': 'Ensolarado', 'Temperatura': 'Quente', 'Umidade': 'Alta', 'Ventoso': 'Falso'}, 'Nao')
({'Tempo': 'Ensolarado', 'Temperatura': 'Quente', 'Umidade': 'Alta', 'Ventoso': 'Verdadeiro'}, 'Nao')
({'Tempo': 'Nublado', 'Temperatura': 'Quente', 'Umidade': 'Alta', 'Ventoso': 'Falso'}, 'Sim')
({'Tempo': 'Chuvoso', 'Temperatura': 'Amena', 'Umidade': 'Alta', 'Ventoso': 'Falso'}, 'Sim')
({'Tempo': 'Chuvoso', 'Temperatura': 'Fria', 'Umidade': 'Normal', 'Ventoso': 'Falso'}, 'Sim')
({'Tempo': 'Chuvoso', 'Temperatura': 'Fria', 'Umidade': 'Normal', 'Ventoso': 'Verdadeiro'}, 'Nao')
({'Tempo': 'Nublado', 'Temperatura': 'Fria', 'Umidade': 'Normal', 'Ventoso': 'Verdadeiro'}, 'Sim')
({'Tempo': 'Ensolarado', 'Temperatura': 'Amena', 'Umidade': 'Alta', 'Ventoso': 'Falso'}, 'Nao')
({'Tempo': 'Ensolarado', 'Temperatura': 'Fria', 'Umidade': 'Normal', 'Ventoso': 'Falso'}, 'Sim')
({'Tempo': 'Chuvoso', 'Temperatura': 'Amena', 'Umidade': 'Normal', 'Ventoso': 'Falso'}, 'Sim')
({'Tempo': 'Ensolarado', 'Temperat

1. Desenvolvimento do algoritmo de indução de uma árvore de decisão, usando como critério de seleção de atributos para divisão de nós o **Ganho de Informação (baseado no conceito de entropia)**. **Tratando tanto atributos categóricos quanto numéricos.**

In [3]:

def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)

    return sum( data_entropy(subset) * len(subset) / total_count for subset in subsets )

def group_by(items, key_fn):
    """returns a defaultdict(list), where each input item is in the list whose key is key_fn(item)"""
    groups = defaultdict(list)
    for item in items:
        key = key_fn(item)
        groups[key].append(item)
    return groups

def partition_by(inputs, attribute):
    """returns a dict of inputs partitioned by the attribute each input is a pair (attribute_dict, label)"""
    return group_by(inputs, lambda x: x[0][attribute])

def partition_entropy_by(inputs, attribute):
    """computes the entropy corresponding to the given partition"""
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values()) # retorna a media dos sub-conjuntos resultantes

def gain(class_entropy, mean_entropy):
    return class_entropy - mean_entropy;

def class_entropy(inputs):
    """retorna a entropia sobre a classe"""
    class_values = []
    for item in inputs:
        class_values.append(item[1]) # suponto a definição atual dos dados!
    probabilities = class_probabilities(class_values)
    class_entropy = entropy(probabilities)
    return class_entropy

def diff_class(inputs):
    classes = []
    for current_class in list(set([label for item, label in inputs if label])):
        classes.append({'class': current_class, 'amount': 0})
    for current_class in [label for item, label in inputs if label]:
        for my_class in classes:
            if(my_class['class']  == current_class):
                my_class['amount'] = my_class['amount'] + 1
    return classes

def highest_class(classes):
    my_class = classes[0]
    for current_class in classes:
        if(current_class['amount'] > my_class['amount']):
            my_class = current_class
    return my_class
    

def build_tree_id3(inputs, split_candidates=None):

    # if this is our first pass,
    # all keys of the first input are split candidates
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
              
    # obtendo as classes
    # [(classe, quantidade)]
    classes = diff_class(inputs)
    print(classes)
    print(highest_class(classes))
    
    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues
    
    # verificando a homogeneidade do cojunto
    if(len(classes) == 1):
        return classes[0]['class']

    if not split_candidates:          # if no split candidates left
        return highest_class(classes) # return the majority leaf

    # otherwise, split on the best attribute
    
    best_gain = 0
    best_attribute = None
    for candidate in split_candidates:
        mean_entropy = partition_entropy_by(inputs, candidate)
        print('[INFO] Atributo: ', candidate, '- entropia média: ', mean_entropy)
        gain_result = gain(class_entropy(inputs), mean_entropy)
        print('[INFO] Atributo: ', candidate, '- ganho: ', gain_result)
        if(gain_result > best_gain):
            best_gain = gain(class_entropy(inputs), mean_entropy)
            best_attribute = candidate
        print('--')
    
    print('----------------------------------------')
    print('[INFO] best_attribute: ', best_attribute)
    print('----------------------------------------')
    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    # recursively build the subtrees
    subtrees = { attribute : build_tree_id3(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    #subtrees[None] = num_trues > num_falses # default case

    return (best_attribute, subtrees)

if __name__ == "__main__":
    # construindo a árvore sobre os dados construindos anteriormente
    tree = build_tree_id3(dataset)
    print('\n')
    print('----------------------------------------')
    print("[INFO] Árvore construida:")
    print('----------------------------------------')
    print(tree)

[{'class': 'Sim', 'amount': 9}, {'class': 'Nao', 'amount': 5}]
{'class': 'Sim', 'amount': 9}
[INFO] Atributo:  Tempo - entropia média:  0.6935361388961919
[INFO] Atributo:  Tempo - ganho:  0.246749819774439
--
[INFO] Atributo:  Temperatura - entropia média:  0.9110633930116763
[INFO] Atributo:  Temperatura - ganho:  0.029222565658954647
--
[INFO] Atributo:  Umidade - entropia média:  0.7884504573082896
[INFO] Atributo:  Umidade - ganho:  0.15183550136234136
--
[INFO] Atributo:  Ventoso - entropia média:  0.8921589282623617
[INFO] Atributo:  Ventoso - ganho:  0.04812703040826927
--
----------------------------------------
[INFO] best_attribute:  Tempo
----------------------------------------
[{'class': 'Sim', 'amount': 2}, {'class': 'Nao', 'amount': 3}]
{'class': 'Nao', 'amount': 3}
[INFO] Atributo:  Temperatura - entropia média:  0.4
[INFO] Atributo:  Temperatura - ganho:  0.5709505944546686
--
[INFO] Atributo:  Umidade - entropia média:  0.0
[INFO] Atributo:  Umidade - ganho:  0.97095

2. Uma função para percorrer a árvore de decisão treinada e realizar a classificação de uma nova instância (do conjunto de teste);

In [14]:
def classify(tree, classes, input):
    """classify the input using the given decision tree"""

    # if this is a leaf node, return its value
    if tree in classes:
        return tree

    # otherwise find the correct subtree
    attribute, subtree_dict = tree

    subtree_key = input.get(attribute)  # None if input is missing attribute

    if subtree_key not in subtree_dict: # if no subtree for key,
        subtree_key = None              # we'll use the None subtree

    subtree = subtree_dict[subtree_key] # choose the appropriate subtree
    return classify(subtree, classes, input)     # and use it to classify the input


# obtendo as classes possiveis para a àrvore
classes = list(set([label for item, label in dataset if label]))

print('\n')
print('[INFO] Dados de amostras (ultimas duas linhas) para teste de resultado:')

print('Nublado / Quente / Normal / Falso ->', classify(tree, classes,
    { 'Tempo' : 'Nublado',
      'Temperatura' : 'Quente',
      'Umidade' : 'Normal',
      'Ventoso' : 'Falso'} ))
print('Chuvoso / Amena / Alta / Verdadeiro ->', classify(tree, classes,
    { 'Tempo' : 'Chuvoso',
      'Temperatura' : 'Amena',
      'Umidade' : 'Alta',
      'Ventoso' : 'Verdadeiro'} ))



[INFO] Dados de amostras (ultimas duas linhas) para teste de resultado:
Nublado / Quente / Normal / Falso -> Sim
Chuvoso / Amena / Alta / Verdadeiro -> Nao


3. O mecanismo de bootstrap (amostragem com reposição) para geração de subconjuntos a partir do conjunto de dados de treinamento originais. Cada bootstrap será utilizado para o treinamento de uma árvore no aprendizado ensemble

4. O mecanismo de amostragem de m atributos a cada divisão de nó, a partir dos quais será selecionado o melhor atributo de acordo com o critério de Ganho de Informação

5. O treinamento de um ensemble de árvores de decisão, adotando os mecanismos de bootstrap e seleção de atributos com amostragem, como mencionados acima

6. O mecanismo de votação majoritária entre as múltiplas árvores de decisão no ensemble, para classificação de novas instâncias utilizando o modelo de Florestas Aleatórias

7. A técnica de validação cruzada (cross-validation) estratificada, para avaliar poder de generalização do modelo e a variação de desempenho de acordo com diferentes valores para os parâmetros do algoritmo (ex., número de árvores no ensemble)

8. Avaliação do impacto do número de árvores no desempenho do ensemble